In [ ]:
import os
import pandas as pd
import glob
import plotly.express as px
import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers import Dense
from keras.models import Model

import tensorflow as tf
from sklearn.metrics import accuracy_score


# Importing Data

In [ ]:
all_files = [i for i in glob.glob('*.{}'.format('csv'))]

#combine all files in the list
combined_df = pd.concat([pd.read_csv(f,encoding='latin-1') for f in all_files])

combined_df.count()

# Data Preprocessing

In [ ]:
combined_df.isnull().values.any()
combined_df.isnull().values.sum()

1182

In [ ]:
combined_df = combined_df.dropna()

In [ ]:
#counting the number of tags
combined_df["total tags"] = combined_df["tags"].apply(lambda text: len(str(text).split("|")))

combined_df.head()

In [ ]:
combined_df.describe()

In [ ]:
main_df = combined_df[['description','tags']]

main_df.head()

In [ ]:
# Replacing all characters that are NOT alphaNumeric or whitespace characters with "" (or nothing)
main_df["tags"] = main_df['tags'].str.replace('[^\w\s]',' ')

main_df["description"] = main_df['description'].str.replace('[^\w\s]',' ')
main_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning:

The default value of regex will change from True to False in a future version.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning:

The default value of regex will change from True to False in a future version.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

,description,tags
0,SHANTELL S CHANNEL https www youtube com s...,SHANtell martin
1,One year after the presidential election John...,last week tonight trump presidency last week ...
2,WATCH MY PREVIOUS VIDEO â n nSUBSCRIBE â º ...,racist superman rudy mancuso king bach ...
3,Today we find out if Link is a Nickelback amat...,rhett and link gmm good mythical morning ...
4,I know it s been a while since we did this sho...,ryan higa higatv nigahiga i dare you ...


In [ ]:
main_df['description']=main_df['description'].str.lower()
main_df['tags'] = main_df['tags'].str.lower()

main_df.head()

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    #text = re.sub(r'[+-]?([0-9]*[.])?[0-9]+', ' ', text)
    return text

In [ ]:
main_df['description'] = main_df['description'].apply(lambda x: clean_text(x)) 
main_df['tags'] = main_df['tags'].apply(lambda x: clean_text(x)) 
main_df.head()

In [ ]:
#remove single characters
main_df['description'] = main_df['description'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
main_df['tags'] = main_df['tags'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')

In [ ]:
#removed stop words
main_df['description'] = main_df['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

main_df['tags'] = main_df['tags'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

main_df.head()

# EDA

In [ ]:


fig = px.histogram(combined_df,x='total tags',
                        title='Count of total tags in movies'
                        )
fig.update_layout(xaxis_title= 'Total tags',yaxis_title='Number of movies')
fig.show()

Around 3823 videos have only 1 tag in its description while 2985 videos have around 4 tags. The maximum number of tags under a video are 78.

# Feature Engineering Techniques

In [ ]:
# split dataset into training and validation set
xtrain, xtest, ytrain, ytest = train_test_split(main_df['description'], main_df['tags'], test_size=0.2, random_state=9)

In [ ]:
ytrain_split = [i.split(" ") for i in ytrain]
ytest_split = [i.split(" ") for i in ytest]

In [ ]:
multilabel_binarizer = MultiLabelBinarizer()

newytrain = multilabel_binarizer.fit_transform(ytrain_split)
#exit()
newytest =  multilabel_binarizer.transform(ytest_split)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning:

unknown class(es) ['166', '2000s', '4k60', 'achmed', 'acoostic', 'aesop', 'aflalo', 'aien', 'allegri', 'amphitheatre', 'ampico', 'anke', 'aoife', 'apologize', 'arian', 'attender', 'auv', 'baio', 'barattender', 'barbies', 'baxter', 'bearding', 'beebee', 'beoplay', 'berzerk', 'bikers', 'billionaires', 'blacklist', 'blackton', 'boomer', 'bq0mxqxmlsk', 'bravest', 'brief', 'brosnahan', 'brushfire', 'bulky', 'byby', 'cambodia', 'cambor', 'cannavale', 'cartoonhangover', 'catchphrase', 'caters', 'cedi', 'cisco', 'cmc', 'cohl', 'companion', 'comprendre', 'comprensiã³n', 'condoms', 'cosmos', 'cresswell', 'cultured', 'cunanan', 'dabbed', 'daemon', 'daenerys', 'daeson', 'dayan', 'decorated', 'definitely', 'dejala', 'detriot', 'diminateurs', 'diminutive', 'diminutives', 'diminutivos', 'dined', 'discontinue', 'dishonest', 'doozy', 'dossi', 'doubts', 'drogo', 'e8', 'eggert', 'elbagir', 'elseworlds', 'embry', 'en

In [ ]:
#newytrain

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

# create TF-IDF features
xtrain_tfidf = tfidf_vectorizer.fit_transform(list(xtrain)).toarray()
xtest_tfidf = tfidf_vectorizer.transform(list(xtest)).toarray()

In [ ]:
#print(newytrain.shape[1])



In [ ]:
#print(xtrain_tfidf)


In [ ]:
#print(xtrain_tfidf.shape)

# Sequential Model

In [ ]:
seq_model = tf.keras.models.Sequential()

In [ ]:
seq_model.add(tf.keras.layers.Dense(1000, input_shape=(10000,), activation='relu'))

seq_model.add(tf.keras.layers.Dense(500, activation='relu'))
seq_model.add(tf.keras.layers.Dense(100, activation='relu'))
seq_model.add(tf.keras.layers.Dense(newytrain.shape[1], activation='sigmoid'))

In [ ]:
seq_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])



seq_model.summary()



In [ ]:
seq_model.fit(xtrain_tfidf, newytrain, epochs=50, batch_size=100, validation_split=0.1, verbose=1)


In [ ]:
# make predictions for test set
y_pred = seq_model.predict(xtest_tfidf)

y_pred = y_pred.round()


In [ ]:
print(y_pred)

In [ ]:


print(multilabel_binarizer.inverse_transform(y_pred))


In [ ]:
print(accuracy_score(newytest, y_pred))